# VRNN
Original paper: A Recurrent Latent Variable Model for Sequential Data (https://arxiv.org/pdf/1506.02216.pdf )

In [32]:
from tqdm import tqdm

import torch
from torch import optim
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, datasets
from tensorboardX import SummaryWriter

batch_size = 256
epochs = 30
seed = 1
torch.manual_seed(seed)

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

In [33]:
# Experiment Setting
# generate MNIST by stacking row images(consider row as time step)
def init_dataset(f_batch_size):
    kwargs = {'num_workers': 1, 'pin_memory': True}
    data_dir = '../data'
    mnist_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Lambda(lambda data: data[0])
    ])
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST(data_dir, train=True, download=True,
                       transform=mnist_transform),
        batch_size=f_batch_size, shuffle=True, **kwargs)
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST(data_dir, train=False, transform=mnist_transform),
        batch_size=f_batch_size, shuffle=True, **kwargs)

    fixed_t_size = 28
    return train_loader, test_loader, fixed_t_size

train_loader, test_loader, t_max = init_dataset(batch_size)

In [34]:
from pixyz.models import Model
from pixyz.losses import KullbackLeibler, StochasticReconstructionLoss
from pixyz.losses import IterativeLoss
from pixyz.distributions import Bernoulli, Normal, Deterministic
from pixyz.utils import print_latex

In [35]:
x_dim = 28
h_dim = 100
z_dim = 64
t_max = x_dim

# feature extraction for x
class Phi_x(nn.Module):
    def __init__(self):
        super(Phi_x, self).__init__()
        self.fc0 = nn.Linear(x_dim, h_dim)

    def forward(self, x):
        return F.relu(self.fc0(x))


# feature extraction for z
class Phi_z(nn.Module):
    def __init__(self):
        super(Phi_z, self).__init__()
        self.fc0 = nn.Linear(z_dim, h_dim)

    def forward(self, z):
        return F.relu(self.fc0(z))

f_phi_x = Phi_x().to(device)
f_phi_z = Phi_z().to(device)

In [36]:
class Generator(Bernoulli):
    '''
    Parameterizes the bernoulli(for MNIST) observation likelihood p(x_t | z_t, h_{t-1})
    '''
    def __init__(self):
        super(Generator, self).__init__(cond_var=["z", "h_prev"], var=["x"])
        self.fc1 = nn.Linear(h_dim + h_dim, h_dim)
        self.fc2 = nn.Linear(h_dim, h_dim)
        self.fc3 = nn.Linear(h_dim, x_dim)
        self.f_phi_z = f_phi_z

    def forward(self, z, h_prev):
        '''
        Given the latent z at a particular time step t and hidden state,
        return the vector of probabilities taht parameterizes the bernoulli distribution
        p(x_t | z_t, h_{t-1})
        
        '''
        h = torch.cat((self.f_phi_z(z), h_prev), dim=-1)
        h = F.relu(self.fc1(h))
        h = F.relu(self.fc2(h))
        return {"probs": torch.sigmoid(self.fc3(h))}

class Prior(Normal):
    '''
    Compared to normal VAE,
    VRNN's Prior for latent z is parameterized by hidden_state h_{t-1}
    z ~ N(loc(h_{t-1}), scale(h_{t-1}))
    '''
    def __init__(self):
        super(Prior, self).__init__(cond_var=["h_prev"], var=["z"])
        self.fc1 = nn.Linear(h_dim, h_dim)
        self.fc21 = nn.Linear(h_dim, z_dim)
        self.fc22 = nn.Linear(h_dim, z_dim)

    def forward(self, h_prev):
        h = F.relu(self.fc1(h_prev))
        return {"loc": self.fc21(h), "scale": F.softplus(self.fc22(h))}

class Inference(Normal):
    '''
    Parameterizes q(z_t | h_{t-1}, x_t)
    infered z ~ N(loc(h_{t-1}, x_t), scale(h_{t-1}, x_t))
    '''
    def __init__(self):
        super(Inference, self).__init__(cond_var=["x", "h_prev"], var=["z"], name="q")
        self.fc1 = nn.Linear(h_dim + h_dim, h_dim)
        self.fc21 = nn.Linear(h_dim, z_dim)
        self.fc22 = nn.Linear(h_dim, z_dim)
        self.f_phi_x = f_phi_x

    def forward(self, x, h_prev):
        h = torch.cat((self.f_phi_x(x), h_prev), dim=-1)
        h = F.relu(self.fc1(h))
        return {"loc": self.fc21(h), "scale": F.softplus(self.fc22(h))}

class Recurrence(Deterministic):
    '''
    RNN for hidden_state
    '''
    def __init__(self):
        super(Recurrence, self).__init__(cond_var=["x", "z", "h_prev"], var=["h"])
        self.rnncell = nn.GRUCell(h_dim * 2, h_dim).to(device)
        self.f_phi_x = f_phi_x
        self.f_phi_z = f_phi_z
        self.hidden_size = self.rnncell.hidden_size

    def forward(self, x, z, h_prev):
        h_next = self.rnncell(torch.cat((self.f_phi_z(z), self.f_phi_x(x)), dim=-1), h_prev)
        return {"h": h_next}

prior = Prior().to(device)
decoder = Generator().to(device)
encoder = Inference().to(device)
recurrence = Recurrence().to(device)

In [37]:
encoder_with_recurrence = encoder * recurrence
generate_from_prior = prior * decoder * recurrence

In [38]:
print_latex(encoder_with_recurrence)

<IPython.core.display.Math object>

In [39]:
print_latex(generate_from_prior)

<IPython.core.display.Math object>

In [40]:
reconst = StochasticReconstructionLoss(encoder_with_recurrence, decoder)
kl = KullbackLeibler(encoder, prior)

# 1 time step loss
step_loss = (reconst + kl).mean()
# Iterative loss for total time step
loss = IterativeLoss(step_loss, max_iter=t_max,
                     series_var=['x'],
                     update_value={"h": "h_prev"})

vrnn = Model(loss, distributions=[encoder, decoder, prior, recurrence],
             optimizer=optim.Adam, optimizer_params={'lr': 1e-3})

print(vrnn)
print_latex(vrnn)

Distributions (for training): 
  q(z|x,h_{prev}), p(x|z,h_{prev}), p(z|h_{prev}), p(h|x,z,h_{prev}) 
Loss function: 
  \sum_{t=1}^{28} mean \left(D_{KL} \left[q(z|x,h_{prev})||p(z|h_{prev}) \right] - \mathbb{E}_{p(h,z|x,h_{prev})} \left[\log p(x|z,h_{prev}) \right] \right) 
Optimizer: 
  Adam (
  Parameter Group 0
      amsgrad: False
      betas: (0.9, 0.999)
      eps: 1e-08
      lr: 0.001
      weight_decay: 0
  )


<IPython.core.display.Math object>

In [41]:
def data_loop(epoch, loader, model, device, train_mode=False):
    mean_loss = 0
    for batch_idx, (data, _) in enumerate(tqdm(loader)):
        data = data.to(device)
        batch_size = data.size()[0]
        x = data.transpose(0, 1)
        h_prev = torch.zeros(batch_size, recurrence.hidden_size).to(device)
        if train_mode:
            mean_loss += model.train({'x': x, 'h_prev': h_prev}).item() * batch_size
        else:
            mean_loss += model.test({'x': x, 'h_prev': h_prev}).item() * batch_size

    mean_loss /= len(loader.dataset)
    if train_mode:
        print('Epoch: {} Train loss: {:.4f}'.format(epoch, mean_loss))
    else:
        print('Test loss: {:.4f}'.format(mean_loss))
    return mean_loss

In [42]:
def plot_image_from_latent(batch_size):
    x = []
    h_prev = torch.zeros(batch_size, recurrence.hidden_size).to(device)
    for step in range(t_max):
        samples = generate_from_prior.sample({'h_prev': h_prev})
        x_t = decoder.sample_mean({"z": samples["z"], "h_prev": samples["h_prev"]})
        h_prev = samples["h"]
        x.append(x_t[None, :])
    x = torch.cat(x, dim=0).transpose(0, 1)
    return x

In [43]:
def generate_image_after_nsteps(n_step_num, original_data):
    xs = []
    x = original_data.transpose(0, 1)
    batch_size = original_data.size()[0]
    h_prev = torch.zeros(batch_size, recurrence.hidden_size).to(device)
    for t in range(28):
        if t < n_step_num - 1:
            # before n_step, reconstruct
            x_t = x[t]
            z_t = encoder.sample_mean({'x': x_t, 'h_prev': h_prev})
            h = recurrence.sample_mean({'x': x_t, 'h_prev': h_prev, 'z': z_t})
            dec_x = decoder.sample_mean({'h_prev': h_prev, 'z': z_t})
            h_prev = h
            xs.append(dec_x[None, :])
        else:
            # generate
            z_t = prior.sample_mean({'h_prev': h_prev})
            dec_x = decoder.sample_mean({'h_prev': h_prev, 'z': z_t})
            h = recurrence.sample_mean({'x': dec_x, 'h_prev': h_prev, 'z': z_t})
            h_prev = h
            xs.append(dec_x[None, :])
    generated_img = torch.cat(xs, dim=0).transpose(0, 1)
    return generated_img

In [44]:
def reconst_image(original_data):
    xs = []
    x = original_data.transpose(0, 1)
    batch_size = original_data.size()[0]
    h_prev = torch.zeros(batch_size, recurrence.hidden_size).to(device)
    for t in range(28):
        # before n_step, reconstruct
        x_t = x[t]
        z_t = encoder.sample_mean({'x': x_t, 'h_prev': h_prev})
        h = recurrence.sample_mean({'x': x_t, 'h_prev': h_prev, 'z': z_t})
        dec_x = decoder.sample_mean({'h_prev': h_prev, 'z': z_t})
        h_prev = h
        xs.append(dec_x[None, :])
    recon_img = torch.cat(xs, dim=0).transpose(0, 1)
    return recon_img

In [45]:
writer = SummaryWriter()
# fixed _x for watching reconstruction improvement
_x, _ = iter(test_loader).next()
_x = _x.to(device)

for epoch in range(1, epochs + 1):
    train_loss = data_loop(epoch, train_loader, vrnn, device, train_mode=True)
    test_loss = data_loop(epoch, test_loader, vrnn, device)

    writer.add_scalar('train_loss', train_loss, epoch)
    writer.add_scalar('test_loss', test_loss, epoch)

    sample = plot_image_from_latent(batch_size)[:, None]
    writer.add_images('Image_from_latent', sample, epoch)
    generated_img_7 = generate_image_after_nsteps(7, _x)
    writer.add_images('Generate_after_7steps', generated_img_7[:, None], epoch)
    
    generated_img_14 = generate_image_after_nsteps(14, _x)
    writer.add_images('Generate_after_14steps', generated_img_14[:, None], epoch)
    
    recon_img = reconst_image(_x)
    writer.add_images('Reconstructed',  recon_img[:, None], epoch)
    
    writer.add_images('orignal', _x[:, None], epoch)



  0%|          | 0/235 [00:00<?, ?it/s]

  0%|          | 1/235 [00:00<01:18,  2.99it/s]

  1%|          | 2/235 [00:00<01:06,  3.52it/s]

  1%|▏         | 3/235 [00:00<00:59,  3.90it/s]

  2%|▏         | 4/235 [00:00<00:54,  4.21it/s]

  2%|▏         | 5/235 [00:01<00:50,  4.60it/s]

  3%|▎         | 6/235 [00:01<00:47,  4.82it/s]

  3%|▎         | 7/235 [00:01<00:44,  5.11it/s]

  3%|▎         | 8/235 [00:01<00:42,  5.29it/s]

  4%|▍         | 9/235 [00:01<00:42,  5.32it/s]

  4%|▍         | 10/235 [00:01<00:41,  5.42it/s]

  5%|▍         | 11/235 [00:02<00:41,  5.42it/s]

  5%|▌         | 12/235 [00:02<00:40,  5.50it/s]

  6%|▌         | 13/235 [00:02<00:41,  5.40it/s]

  6%|▌         | 14/235 [00:02<00:41,  5.36it/s]

  6%|▋         | 15/235 [00:02<00:41,  5.34it/s]

  7%|▋         | 16/235 [00:03<00:41,  5.31it/s]

  7%|▋         | 17/235 [00:03<00:41,  5.31it/s]

  8%|▊         | 18/235 [00:03<00:40,  5.31it/s]

  8%|▊         | 19/235 [00:03<00:40,  5.30it/s]

  9%|▊         |

 62%|██████▏   | 145/235 [00:26<00:15,  5.92it/s]

 62%|██████▏   | 146/235 [00:26<00:15,  5.93it/s]

 63%|██████▎   | 147/235 [00:26<00:14,  5.93it/s]

 63%|██████▎   | 148/235 [00:26<00:14,  5.92it/s]

 63%|██████▎   | 149/235 [00:27<00:14,  5.91it/s]

 64%|██████▍   | 150/235 [00:27<00:14,  5.91it/s]

 64%|██████▍   | 151/235 [00:27<00:14,  5.91it/s]

 65%|██████▍   | 152/235 [00:27<00:14,  5.92it/s]

 65%|██████▌   | 153/235 [00:27<00:13,  5.93it/s]

 66%|██████▌   | 154/235 [00:27<00:14,  5.66it/s]

 66%|██████▌   | 155/235 [00:28<00:14,  5.48it/s]

 66%|██████▋   | 156/235 [00:28<00:14,  5.58it/s]

 67%|██████▋   | 157/235 [00:28<00:13,  5.68it/s]

 67%|██████▋   | 158/235 [00:28<00:13,  5.76it/s]

 68%|██████▊   | 159/235 [00:28<00:13,  5.82it/s]

 68%|██████▊   | 160/235 [00:28<00:12,  5.87it/s]

 69%|██████▊   | 161/235 [00:29<00:12,  5.90it/s]

 69%|██████▉   | 162/235 [00:29<00:12,  5.94it/s]

 69%|██████▉   | 163/235 [00:29<00:12,  5.95it/s]

 70%|██████▉   | 164/235 [00:29

Epoch: 1 Train loss: 255.8023




  2%|▎         | 1/40 [00:00<00:07,  5.41it/s]

  8%|▊         | 3/40 [00:00<00:05,  6.43it/s]

 10%|█         | 4/40 [00:00<00:05,  7.06it/s]

 12%|█▎        | 5/40 [00:00<00:04,  7.59it/s]

 15%|█▌        | 6/40 [00:00<00:04,  8.01it/s]

 18%|█▊        | 7/40 [00:00<00:03,  8.32it/s]

 22%|██▎       | 9/40 [00:00<00:03,  9.10it/s]

 28%|██▊       | 11/40 [00:01<00:02,  9.72it/s]

 32%|███▎      | 13/40 [00:01<00:02, 10.22it/s]

 38%|███▊      | 15/40 [00:01<00:02, 10.60it/s]

 42%|████▎     | 17/40 [00:01<00:02, 10.90it/s]

 48%|████▊     | 19/40 [00:01<00:01, 11.10it/s]

 52%|█████▎    | 21/40 [00:02<00:01, 11.26it/s]

 57%|█████▊    | 23/40 [00:02<00:01, 11.38it/s]

 62%|██████▎   | 25/40 [00:02<00:01, 11.45it/s]

 68%|██████▊   | 27/40 [00:02<00:01, 11.12it/s]

 72%|███████▎  | 29/40 [00:02<00:00, 11.26it/s]

 78%|███████▊  | 31/40 [00:02<00:00, 11.32it/s]

 82%|████████▎ | 33/40 [00:03<00:00, 11.36it/s]

 88%|████████▊ | 35/40 [00:03<00:00, 11.42it/s]

 92%|█████████▎| 37/40 [0

Test loss: 223.2199


...



  0%|          | 0/235 [00:00<?, ?it/s]

  0%|          | 1/235 [00:00<01:12,  3.23it/s]

  1%|          | 2/235 [00:00<01:02,  3.73it/s]

  1%|▏         | 3/235 [00:00<00:55,  4.17it/s]

  2%|▏         | 4/235 [00:00<00:51,  4.48it/s]

  2%|▏         | 5/235 [00:01<00:49,  4.69it/s]

  3%|▎         | 6/235 [00:01<00:45,  5.00it/s]

  3%|▎         | 7/235 [00:01<00:43,  5.23it/s]

  3%|▎         | 8/235 [00:01<00:41,  5.41it/s]

  4%|▍         | 9/235 [00:01<00:40,  5.53it/s]

  4%|▍         | 10/235 [00:01<00:39,  5.63it/s]

  5%|▍         | 11/235 [00:02<00:39,  5.70it/s]

  5%|▌         | 12/235 [00:02<00:38,  5.73it/s]

  6%|▌         | 13/235 [00:02<00:38,  5.77it/s]

  6%|▌         | 14/235 [00:02<00:38,  5.79it/s]

  6%|▋         | 15/235 [00:02<00:37,  5.81it/s]

  7%|▋         | 16/235 [00:02<00:37,  5.82it/s]

  7%|▋         | 17/235 [00:03<00:37,  5.84it/s]

  8%|▊         | 18/235 [00:03<00:37,  5.85it/s]

  8%|▊         | 19/235 [00:03<00:38,  5.64it/s]

  9%|▊         |

 62%|██████▏   | 145/235 [00:25<00:14,  6.03it/s]

 62%|██████▏   | 146/235 [00:25<00:14,  6.07it/s]

 63%|██████▎   | 147/235 [00:26<00:14,  6.08it/s]

 63%|██████▎   | 148/235 [00:26<00:14,  6.06it/s]

 63%|██████▎   | 149/235 [00:26<00:14,  5.77it/s]

 64%|██████▍   | 150/235 [00:26<00:15,  5.60it/s]

 64%|██████▍   | 151/235 [00:26<00:15,  5.51it/s]

 65%|██████▍   | 152/235 [00:27<00:15,  5.37it/s]

 65%|██████▌   | 153/235 [00:27<00:14,  5.57it/s]

 66%|██████▌   | 154/235 [00:27<00:14,  5.49it/s]

 66%|██████▌   | 155/235 [00:27<00:14,  5.44it/s]

 66%|██████▋   | 156/235 [00:27<00:14,  5.40it/s]

 67%|██████▋   | 157/235 [00:27<00:14,  5.38it/s]

 67%|██████▋   | 158/235 [00:28<00:14,  5.36it/s]

 68%|██████▊   | 159/235 [00:28<00:14,  5.35it/s]

 68%|██████▊   | 160/235 [00:28<00:14,  5.34it/s]

 69%|██████▊   | 161/235 [00:28<00:13,  5.34it/s]

 69%|██████▉   | 162/235 [00:28<00:13,  5.33it/s]

 69%|██████▉   | 163/235 [00:29<00:13,  5.32it/s]

 70%|██████▉   | 164/235 [00:29

Epoch: 30 Train loss: 174.6337




  2%|▎         | 1/40 [00:00<00:07,  5.47it/s]

  8%|▊         | 3/40 [00:00<00:05,  6.59it/s]

 12%|█▎        | 5/40 [00:00<00:04,  7.70it/s]

 15%|█▌        | 6/40 [00:00<00:04,  8.27it/s]

 20%|██        | 8/40 [00:00<00:03,  9.08it/s]

 25%|██▌       | 10/40 [00:00<00:03,  9.76it/s]

 30%|███       | 12/40 [00:01<00:02, 10.31it/s]

 35%|███▌      | 14/40 [00:01<00:02, 10.74it/s]

 40%|████      | 16/40 [00:01<00:02, 11.06it/s]

 45%|████▌     | 18/40 [00:01<00:02, 10.52it/s]

 50%|█████     | 20/40 [00:01<00:01, 10.16it/s]

 55%|█████▌    | 22/40 [00:02<00:01,  9.93it/s]

 60%|██████    | 24/40 [00:02<00:01,  9.78it/s]

 62%|██████▎   | 25/40 [00:02<00:01,  9.64it/s]

 65%|██████▌   | 26/40 [00:02<00:01,  9.55it/s]

 68%|██████▊   | 27/40 [00:02<00:01,  9.48it/s]

 70%|███████   | 28/40 [00:02<00:01,  9.44it/s]

 75%|███████▌  | 30/40 [00:02<00:01,  9.88it/s]

 80%|████████  | 32/40 [00:03<00:00, 10.41it/s]

 85%|████████▌ | 34/40 [00:03<00:00, 10.82it/s]

 90%|█████████ | 36/40 

Test loss: 173.8075
